In [17]:
import os.path as osp
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid, Coauthor, Amazon
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GAE, VGAE, APPNP
import torch_geometric.transforms as T

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, default='VGNAE')
parser.add_argument('--dataset', type=str, default='Cora')
parser.add_argument('--epochs', type=int, default=300)
parser.add_argument('--channels', type=int, default=128)
parser.add_argument('--scaling_factor', type=float, default=1.8)
parser.add_argument('--training_rate', type=float, default=0.8) 
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--dataset DATASET]
                             [--epochs EPOCHS] [--channels CHANNELS]
                             [--scaling_factor SCALING_FACTOR]
                             [--training_rate TRAINING_RATE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\caleb\AppData\Roaming\jupyter\runtime\kernel-e6d80757-d4f1-463e-8069-96629262bd82.json


SystemExit: 2

C:\Users\caleb\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [31]:
!python main.py --dataset=Cora --training_rate=0.2 --epochs=300

Epoch: 001, LOSS: 10.9415, AUC: 0.6472, AP: 0.6925
Epoch: 002, LOSS: 10.2842, AUC: 0.6504, AP: 0.6950
Epoch: 003, LOSS: 10.6782, AUC: 0.6531, AP: 0.6971
Epoch: 004, LOSS: 10.6574, AUC: 0.6551, AP: 0.6986
Epoch: 005, LOSS: 10.6286, AUC: 0.6570, AP: 0.7001
Epoch: 006, LOSS: 10.1641, AUC: 0.6590, AP: 0.7016
Epoch: 007, LOSS: 10.0778, AUC: 0.6611, AP: 0.7031
Epoch: 008, LOSS: 10.1770, AUC: 0.6631, AP: 0.7045
Epoch: 009, LOSS: 10.1049, AUC: 0.6653, AP: 0.7061
Epoch: 010, LOSS: 9.9055, AUC: 0.6677, AP: 0.7078
Epoch: 011, LOSS: 9.7274, AUC: 0.6703, AP: 0.7096
Epoch: 012, LOSS: 8.9727, AUC: 0.6730, AP: 0.7116
Epoch: 013, LOSS: 8.9809, AUC: 0.6760, AP: 0.7139
Epoch: 014, LOSS: 8.8456, AUC: 0.6793, AP: 0.7163
Epoch: 015, LOSS: 8.6326, AUC: 0.6832, AP: 0.7192
Epoch: 016, LOSS: 9.0088, AUC: 0.6876, AP: 0.7225
Epoch: 017, LOSS: 8.7783, AUC: 0.6930, AP: 0.7265
Epoch: 018, LOSS: 8.3972, AUC: 0.6993, AP: 0.7310
Epoch: 019, LOSS: 8.4984, AUC: 0.7071, AP: 0.7369
Epoch: 020, LOSS: 7.6068, AUC: 0.7165, AP

Processing...
Done!
C:\Users\caleb\anaconda3\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [46]:
# Load the Cora dataset
dataset = Planetoid(root='/tmp/cora', name='Cora')

In [47]:
data = dataset[0]
data = T.NormalizeFeatures()(data)

In [48]:
model_used = 'VGNAE'
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, edge_index):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(in_channels, out_channels)
        self.linear2 = nn.Linear(in_channels, out_channels)
        self.propagate = APPNP(K=1, alpha=0)

    def forward(self, x, edge_index,not_prop=0):
        if model_used  == 'GNAE':
            x = self.linear1(x)
            x = F.normalize(x,p=2,dim=1)  * args.scaling_factor
            x = self.propagate(x, edge_index)
            return x

        if model_used  == 'VGNAE':
            x_ = self.linear1(x)
            x_ = self.propagate(x_, edge_index)

            x = self.linear2(x)
            x = F.normalize(x,p=2,dim=1) * args.scaling_factor
            x = self.propagate(x, edge_index)
            return x, x_

        return x

In [49]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
channels = 128
train_rate = 0.8
val_ratio = (1-train_rate) / 3
test_ratio = (1-train_rate) / 3 * 2
data = train_test_split_edges(data.to(dev), val_ratio=val_ratio, test_ratio=test_ratio)

In [50]:
N = int(data.x.size()[0])
if model_used  == 'GNAE':   
    model = GAE(Encoder(data.x.size()[1], channels, data.train_pos_edge_index)).to(dev)
if model_used  == 'VGNAE':
    model = VGAE(Encoder(data.x.size()[1], channels, data.train_pos_edge_index)).to(dev)

data.train_mask = data.val_mask = data.test_mask = data.y = None
x, train_pos_edge_index = data.x.to(dev), data.train_pos_edge_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [51]:
def train():
    model.train()
    optimizer.zero_grad()
    z  = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    if model_used  in ['VGAE']:
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss

In [52]:
def test(pos_edge_index, neg_edge_index, plot_his=0):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [53]:
epochs = 300
for epoch in range(1,epochs):
    loss = train()
    loss = float(loss)
    
    with torch.no_grad():
        test_pos, test_neg = data.test_pos_edge_index, data.test_neg_edge_index
        auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
        print('Epoch: {:03d}, LOSS: {:.4f}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, loss, auc, ap))

NameError: name 'args' is not defined

In [42]:
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import VGAE
import torch.nn.functional as F

# Split the graph into training and test sets
train_pos_edge_index, test_pos_edge_index = train_test_split_edges(data.edge_index)

# Create a new Data object with the positive edges only
train_data = data.clone()
train_data.edge_index = train_pos_edge_index

# Define VGAE model and optimizer
model = VGAE(data.num_features, 16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    adj_recon = model.decode(z, train_data.edge_index)
    loss = F.binary_cross_entropy(adj_recon.view(-1), train_data.adjacency_matrix().view(-1))
    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss

epochs = 300
for epoch in range(1,epochs):
    loss = train()
    loss = float(loss)

# Evaluate the model on the test set
model.eval()
z = model.encode(data.x, data.edge_index)
test_pred = (z[test_pos_edge_index[0]] * z[test_pos_edge_index[1]]).sum(dim=1)
test_true = torch.ones_like(test_pred)
test_ap = sklearn.metrics.average_precision_score(test_true, test_pred.detach().numpy())
test_auc = sklearn.metrics.roc_auc_score(test_true, test_pred.detach().numpy())

print(f"Test AP: {test_ap:.4f}")
print(f"Test AUC: {test_auc:.4f}")

TypeError: argument of type 'NoneType' is not iterable